# Assignment  Practice Text classification with Naive Bayes 

## Notebook made by   (If not filled in correctly: 0 pts for assignment)

__Name(s)__: Samar Hashemi , Pepijn Groenen

__Student id(s)__ : 11045035 , 11015802

### Pledge (taken from [Coursera's Honor Code](https://www.coursera.org/about/terms/honorcode) )



Put here a selfie with your photo where you hold a signed paper with the following text: (if this is team work, put two selfies here). The link must be to some place on the web, not to a local file. **Assignments without the selfies will not be graded and receive 0 points.**

> My answers to homework, quizzes and exams will be my own work (except for assignments that explicitly permit collaboration).

>I will not make solutions to homework, quizzes or exams available to anyone else. This includes both solutions written by me, as well as any official solutions provided by the course staff.

>I will not engage in any other activities that will dishonestly improve my results or dishonestly improve/hurt the results of others.

<img src='link to your selfie'/> https://www.sharelatex.com/project/59b5c3d3d84b040e4adb8419

# Practice Text classification with Naive Bayes  
        
        
        
<h3>Abstract</h3>
<p>We will do text classification on a collection of Dutch parliamentary questions.
    The website <a href="https://zoek.officielebekendmakingen.nl/zoeken/parlementaire_documenten">officielebekendmakingen.nl</a>lets you search in "kamervragen".
    You can donwload
    <a href='http://data.politicalmashup.nl/kamervragen/PoliDocs_Kamervragen.zip'>this zipfile with Kamervragen in XML</a>
    to see some of the  data in XML format. 
    It also contains style sheets to show the XML well in a browser.  
    The <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/'>MYSQL directory</a> contains an <a href='http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR14807.xml'>example   Kamervraag XML file</a> and a file `kvr.csv.gz` with all those kamervragen in a handy csv format. Note that in your browser you see the result of applying stylesheets. So choose View Source or open it in an editor.</p>

<h3>First exploration</h3>

See below.

<h2>Exercise</h2>

<p>We will use the fields in elements of the form <tt> &lt;item attribuut="Afkomstig_van"></tt> as our classes. 
    These are the ministeries to whom the question is addressed.
    An example is 
    <pre>
        &lt;item attribuut="Afkomstig_van">Landbouw, Natuurbeheer en Visserij (LNV)&lt;/item>
    </pre>
    Note that these labels are <strong>not normalized</strong>, see e.g. the counts below:
    <pre>
Justitie (JUS)                                                   3219
Volksgezondheid, Welzijn en Sport (VWS)                          2630
Buitenlandse Zaken (BUZA)                                        1796
Verkeer en Waterstaat (VW)                                       1441
Justitie                                                         1333
Sociale Zaken en Werkgelegenheid (SZW)                           1231
Onderwijs, Cultuur en Wetenschappen (OCW)                        1187
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer (VROM)     984
FinanciÃ«n (FIN)                                                   960
Volksgezondheid, Welzijn en Sport                                 951
Economische Zaken (EZ)                                            946
Buitenlandse Zaken                                                753
Binnenlandse Zaken en Koninkrijksrelaties (BZK)                   725
Verkeer en Waterstaat                                             724
Defensie (DEF)                                                    646
Sociale Zaken en Werkgelegenheid                                  607
Landbouw, Natuurbeheer en Visserij (LNV)                          586
Volkshuisvesting, Ruimtelijke Ordening en Milieubeheer            554
Onderwijs, Cultuur en Wetenschappen                               532
Vreemdelingenzaken en Integratie (VI)                             466
    </pre>
</p>

  <ol>
      <li>Normalize the values for "ministerie" and choose 10 ministeries to work with. </li>
      <li>Implement the two algorithms in Fig MRS.13.2, using your earlier code for creating term and document frequencies.
      It might be easier to use the representation and formula given in MRS section 13.4.1.</li>
      <li>On this collection, train NB text classifiers for 10 different classes with enough and interesting data.</li>
      <li>Compute for each term and each of your 10 classes its utility for that class using mutual information.</li>
      <li>For each class, show the top 10 words as in Figure 13.7 in MRS.</li>
      <li>Evaluate your classifiers using Precision, Recall and F1. (
           <br/>
          Give a table in which you show these values for using the top 10, top 100 terms and all terms, for all of your 10 classes.
          Thus do feature selection per class, and use for each class the top n best features for that class. 
          <br/>
      Also show the microaverage(s) for all 10 classes together.
      <br/>
      If you like you can also present this in a figure like MRS.13.8. 
      Then compute the F1 measure for the same number of terms as in that figure.</li>
      <li>Reflect and report briefly about your choices in this process and about the obtained results. </li>
  </ol>

<h3>Training/Testing</h3>
<p>It is important that you do not test your classifier using documents that have also been used in training.
    So split up your collection in a training set and a test set. A 80%-20% split is reasonable.

<br/>
    If you have too little data you can use 5 or <a href="http://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">10-fold cross validation</a>.</p>

<h2>Form of presentation</h2>
<ul>
    <li>Make slides or wikipages and have your system running (this could just be an IPython notebook with a classify function that accepts any string and classifies it.) ~~and be able to accept documents from the web.~~ </li>
    <li>Create one or two slides or wikipages for each of the sub exercises listed above.
</li>
<li>Make it clear in the heading of the slides which sub exercises you talk about.</li>
    <li>Show running code with one or two  good examples (a TP of course, but also a FP and an error-analysis is nice to show). </li>

</ul>

<h2>Form of handing in your final product</h2>
<ul>
    <li>An IPython notebook would be perfect, with clear indications which part of the code answers which subquestion.</li>
    <li>A clear git repo, with good comments and a clear separation and indication what code does what is also fine.</li>
    <li> You are free to program in whatever language you prefer.</li>
</ul>


# Some example code

In [1]:
import pandas as pd

# Change to KVR1000.csv.gz if this becomes to0 slow for you
kvrdf= pd.read_csv('http://maartenmarx.nl/teaching/zoekmachines/LectureNotes/MySQL/KVR.csv.gz', 
                   compression='gzip', sep='\t', encoding='utf-8',
                   index_col=0, names=['jaar', 'partij','titel','vraag','antwoord','ministerie']) 
print kvrdf.shape
kvrdf.head(20)

(40516, 6)


,jaar,partij,titel,vraag,antwoord,ministerie
KVR1000.xml,1994,PvdA,De vragen betreffen de betrouwbaarheid van de...,Hebt u kennisgenomen van het televisieprogram...,Ja. Het bedoelde geluidmeetpunt is eigendom v...,Verkeer en Waterstaat
KVR10000.xml,1999,PvdA,Vragen naar aanleiding van berichten (uitzend...,Kent u de berichten over de situatie in de Me...,,Justitie
KVR10001.xml,1999,SP,"Vragen naar aanleiding van de berichten ""Nede...",Kent u de berichten «Nederland steunt de Soeh...,,Financien
KVR10002.xml,1999,PvdA,Vragen over de gebrekkige opvang van verpleeg...,Kent u het bericht over onderzoek van Nu91 me...,Ja. Het onderzoek van NU’91 wijst uit dat het...,"Volksgezondheid, Welzijn en Sport"
KVR10003.xml,1999,PvdA,Vragen over onbetrouwbaarheid van filemeldingen.,Hebt u kennisgenomen van de berichten over de...,Ja. Nee. Door de waarnemers van het Algemeen ...,Verkeer en Waterstaat
KVR10004.xml,1999,D66,Vragen naar aanleiding van het bericht in de ...,Kent u het bericht als zou het RIVM onjuiste ...,Ja. Nee. Als reactie op het Volkskrant-artike...,Verkeer en Waterstaat
KVR10006.xml,1999,PvdA,,Kent u berichten over asbestgevaar voor KFOR-...,"Kamerstuk 22 181, nr. 293",Defensie
KVR10007.xml,1999,D66,Vragen naar aanleiding van het artikel ''Eila...,Kent u het artikel Eilander artsen vechten vo...,"Ja. Ik ben van mening, en ga er vanuit dat di...","Volksgezondheid, Welzijn en Sport"
KVR10008.xml,1999,GroenLinks,Vragen over de mogelijke terugkeer naar het o...,Kent u het artikel «De specialist declareert ...,"Ja. Neen, op dit moment is in drie van de rui...","Volksgezondheid, Welzijn en Sport (VWS)"
KVR10009.xml,1999,SP,Vragen over de uiteindelijke invulling van ui...,Is over de uiteindelijke invulling van de uit...,"De Ziekenfondsraad, thans College voor zorgve...","Volksgezondheid, Welzijn en Sport (VWS)"


In [2]:
from collections import Counter,defaultdict
from bs4 import BeautifulSoup
import os
import glob
import math
import nltk
from __future__ import division
import operator
import numpy as np

In [12]:
#      \   
# 8:>)=[]==|
#      / 

import pickle
# list of 10 ministeries, named by a string that is present is all/most instances of this name
ministries = ["Justitie", "Volksgezondheid, Welzijn en Sport", "Buitenlandse Zaken", "Verkeer en Waterstaat", "Sociale Zaken en Werkgelegenheid", "Onderwijs, Cultuur en Wetenschap", "Finan", "Economische Zaken", "Defensie", "Binnenlandse Zaken"]
     
# This function can save an object in local memory
def save_obj(obj, name ):
    cwd = os.getcwd() + '\\'
    with open( cwd + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        

# This function can load an object from memory        
def load_obj(name ):
    cwd = os.getcwd() + '\\'
    with open( cwd + name + '.pkl', 'rb') as f:
        return pickle.load(f)


# This function is used to make an dict that contains 10 ministries that are mapped to a list of indexes of items in 
# the kvrdf object that belong to a certain ministry. This dict is than saved in the working dictionarry under the name : 
# 'ministry_indexes'.
def normalize():
    norm = dict()
    for ministry in ministries:
        norm[ministry] = []
    for i in range(0,len(kvrdf.ministerie)):
        if (i % 100 == 0):
            #save_obj(norm, 'ministry_indexes')
            print i
        if type(kvrdf.ministerie[i]) == unicode:
            for ministry in ministries:
                if ministry in kvrdf.ministerie[i]:
                    norm[ministry].append(i)
    save_obj(norm, 'ministry_indexes')
    return 1

# This loads the ministry indexes dictionary from memory.                 
ministry_indexes = load_obj('ministry_indexes')





In [22]:
nltk.download('stopwords')
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import names, stopwords, words
DutchStop= stopwords.words('dutch')

# Join all titels of the questions together

              
# This function creates a BoW with a counter for each word. There are several options: if the indexes variable is None, all 
# items in the kvrdf will be red. Otherwise only the indexes in "indexes". Also, the BoW can contain all words from 1 or 2 
# collums in the kvrdf object. If there is just one, this should be in kv_collum1 and boolean should be on False. If there are 2,
# boolean should be True. 
#
# Examples: 
#
# This returns a BoW of all words in titel en vraag of the Justitie ministry:
#
# Create_BoW(ministry_indexes['Justitie'], kvrdf.titel, kvrdf.vraag, True)
#
# This returns a BoW oall words in titel:
#
# Create_BoW(None, kvrdf.titel, None, False)

def Create_BoW(indexes, kv_collum1 , kv_collum2, boolean):
    melt_lijst = []
              
    if (indexes == None):
        for item in kv_collum1:
            if (type(item) == unicode):
                melt_lijst.append(item)
        if boolean:
            for item in kv_collum2:
                if (type(item) == unicode):
                    melt_lijst.append(item)
    else:
        for index in indexes:
            item = kv_collum1[index]
            if (type(item) == unicode):
                melt_lijst.append(item)
        if boolean:
            for index in indexes:
                item = kv_collum2[index]
                if (type(item) == unicode):
                    melt_lijst.append(item)
    
    melt = '\n'.join(melt_lijst)
    BoW= Counter(nltk.word_tokenize(melt))
    BoW= BoW= Counter([w for w in nltk.word_tokenize(melt) if not w in set(DutchStop)])
    return BoW
    


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pepijn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[(u'?', 44713), (u',', 40421), (u'.', 12163), (u'Zo', 7809), (u')', 6085), (u'(', 6069), (u'Is', 4516), (u'Bent', 4394), (u'neen', 3573), (u'welke', 3518), (u'waarom', 3467), (u'Vragen', 3400), (u'mening', 3124), (u'bereid', 3063), (u'Nederland', 3029), (u'Wat', 2829), (u'aanleiding', 2726), (u'Hoe', 2568), (u'Deelt', 2537), (u'vragen', 2502)]


In [30]:
# This function calculates the BoW's for titel, vraag and both titel and vraag. It then saves these objects under the name:
# BoW_collumn_ministry
#
# DO NOT RUN THIS BECAUSE IT WILL OVERRIDE PREVIOUSLY SAVED DATA
def save_BoW():
    save_obj(Create_BoW(None, kvrdf.titel, None, False), 'BoW_titel_all')
    for ministry in ministries: 
        save_obj(Create_BoW(ministry_indexes[ministry], kvrdf.titel, None, False), "BoW_titel_" + ministry)
    save_obj(Create_BoW(None, kvrdf.vraag, None, False), 'BoW_vraag_all')
    for ministry in ministries: 
        save_obj(Create_BoW(ministry_indexes[ministry], kvrdf.vraag, None, False), "BoW_vraag_" + ministry)
    save_obj(Create_BoW(None, kvrdf.vraag, kvrdf.titel, True), 'BoW_both_all')
    for ministry in ministries: 
        save_obj(Create_BoW(ministry_indexes[ministry], kvrdf.vraag, kvrdf.titel, True), "BoW_both_" + ministry)


In [7]:
#      \   
# 8:>)=[]==|
#      / 
import numpy as np
from __future__ import division

# This function trains a multinomial Naive Bayes Classifier. It uses at 1 smoothing. The conditional probabilities and prior 
# probabilities are then saved under the name : "prior_" + collumn. This collumn  is the collumn in the kvrdf object that you want
# the classificaiton to be build upon. It schould be initiated and there are three options:  "vraag", "titel" or "both". 
# Exmple: 
#
# This saves the prior and cond prob of NB based on the "titel" collumn.
#
# train_multinomial_NB(ministry_indexes, ministries, "titel")

def train_multinomial_NB(min_indexes, ministries, collumn):
    words_corpus = load_obj('BoW_' + collumn + '_all')
    n = len(kvrdf.vraag)
    condprob = np.zeros((len(words_corpus), len(ministries)))
    prior = np.zeros(len(ministries))
    for i in range(0, len(ministries)):
        ministry = ministries[i]
        class_indexes = min_indexes[ministry]
        n_c = len(class_indexes)
        prior[i] = n_c / n 
        words_c = load_obj('BoW_' + collumn + '_' + ministry)
        words_in_class = sum(words_c.values()) + len(words_c) # add one smoothing
        print words_in_class  
        for j in range(0, len(words_corpus)):
            key = words_corpus.keys()[j]
            condprob[j][i] = (words_c[key] + 1) / words_in_class # add one smoothing
    save_obj(condprob, "condprob_" + collumn)
    save_obj(prior, "prior_" + collumn)
    return 1


In [ ]:
train_multinomial_NB(ministry_indexes, ministries, "titel")

In [6]:
train_multinomial_NB(ministry_indexes, ministries, "vraag")

651688
541364
398847
289151
247385
256216
213734
198690
149576
246964


1

In [8]:
train_multinomial_NB(ministry_indexes, ministries, "both")

765251
625711
461266
333796
287908
302151
248889
230813
174562
288824


1

In [17]:
#      \   
# 8:>)=[]==|
#      / 

x = load_obj('condprob_titel')
x[1][9]
    
    

2.0533037657591063e-05

In [29]:
def minesteryIndex(minestery):
    count = 0
    for mins in ministries:
        if minestery == mins:
            return count
        elif minestery != mins:
            count += 1
    if count != 0:
        return count

In [23]:
xx = load_obj('prior_titel')
xxx = load_obj('condprob_titel')

In [49]:
def applyMultinomialNB(C,prior1,condprob,collumn,ministry):
    V = load_obj('BoW_' + collumn + '_' + ministry)
    minestery_location = minesteryIndex(ministry)
    score = dict()
    for mins in C:
        index = minesteryIndex(mins)
        score[mins] = load_obj('prior_' + collumn)[index]
        for i in range(len(V)):
            score[mins] += condprob[i][index]
    return max(score.iteritems(), key=operator.itemgetter(1))

applyMultinomialNB(ministry_indexes, xx,xxx, 'titel', 'Justitie' )

('Defensie', 0.65364410396043371)

In [13]:
ministry_indexes['Justitie']

[1,
 16,
 21,
 38,
 39,
 41,
 53,
 59,
 69,
 84,
 85,
 113,
 120,
 145,
 169,
 174,
 176,
 199,
 203,
 204,
 208,
 228,
 237,
 246,
 247,
 256,
 258,
 271,
 272,
 273,
 296,
 302,
 303,
 305,
 308,
 312,
 318,
 321,
 323,
 324,
 327,
 328,
 333,
 335,
 336,
 337,
 339,
 340,
 341,
 350,
 353,
 354,
 357,
 359,
 360,
 365,
 372,
 379,
 392,
 396,
 403,
 411,
 418,
 420,
 421,
 427,
 428,
 429,
 445,
 460,
 461,
 468,
 474,
 475,
 477,
 479,
 484,
 485,
 492,
 496,
 497,
 506,
 511,
 518,
 519,
 520,
 527,
 546,
 580,
 581,
 588,
 589,
 595,
 600,
 602,
 615,
 618,
 620,
 621,
 623,
 625,
 626,
 627,
 631,
 632,
 634,
 642,
 647,
 650,
 658,
 695,
 697,
 698,
 712,
 717,
 723,
 724,
 728,
 735,
 736,
 737,
 740,
 743,
 745,
 760,
 763,
 765,
 766,
 768,
 771,
 772,
 774,
 791,
 792,
 795,
 797,
 804,
 808,
 809,
 811,
 817,
 819,
 824,
 826,
 841,
 842,
 861,
 863,
 867,
 871,
 872,
 875,
 882,
 883,
 884,
 900,
 901,
 904,
 910,
 911,
 915,
 957,
 959,
 963,
 985,
 986,
 988,
 989,
 995

In [11]:
print len(load_obj('condprob_titel'))

77328


In [2]:
#      \   
# /:>)=[]==|
#      / 




40516
 Vragen naar aanleiding van berichten (uitzending NOVA 17 juli 1999) over onderzoeken van de Rijksrecherche waaruit blijt dat er twijfels bestaan over het professioneel gedrag van vier medewerkers van de kliniek.
